In [9]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import subprocess

In [35]:
train_results_dir = os.getcwd() + "/../002-Training-models/train_results/"
!ls {train_results_dir}

model_yolov5_tf_no_imgsz_800_epochs_100_batch_16_dataset_v1_loss_SGD_lr_01
model_yolov5_tf_no_imgsz_800_epochs_50_batch_16_dataset_v1_loss_SGD_lr_01
model_yolov5_tf_yolov5s_imgsz_640_epochs_100_batch_16_dataset_v1_loss_SGD_lr_001
model_yolov5_tf_yolov5s_imgsz_640_epochs_100_batch_16_dataset_v1_loss_SGD_lr_01
model_yolov5_tf_yolov5s_imgsz_640_epochs_50_batch_16_dataset_v1_loss_SGD_lr_01
model_yolov5_tf_yolov5s_imgsz_800_epochs_100_batch_16_dataset_v1_loss_SGD_lr_01
model_yolov5_tf_yolov5s_imgsz_800_epochs_50_batch_16_dataset_v1_loss_SGD_lr_01
model_yolov7_tf_yolov7training_imgsz_640_epochs_100_batch_16_dataset_v1_loss_SGD_lr_001
model_yolov7_tf_yolov7training_imgsz_640_epochs_100_batch_16_dataset_v1_loss_SGD_lr_01
model_yolov7_tf_yolov7training_imgsz_640_epochs_50_batch_16_dataset_v1_loss_SGD_lr_001
model_yolov7_tf_yolov7training_imgsz_800_epochs_100_batch_8_dataset_v1_loss_SGD_lr_01
model_yolov8_tf_no_imgsz_800_epochs_100_batch_16_dataset_v1_loss_SGD_lr_01
model_yolov8_tf_no_imgsz_800_

In [36]:
eval_results_dir = os.getcwd()
dataset_dir = os.getcwd()+"/../datasets/"
output_file = "logs/eval_vs_original_dataset.txt"

In [12]:
# Download YOLOv8
!pip install ultralytics

from IPython import display
display.clear_output()

%cd {eval_results_dir}
!mkdir yolov8

from ultralytics import YOLO
print('YOLOv8 setup complete.')

/Users/florenciavela/Desktop/flor/git/gun-detection/003-Evaluating-models
mkdir: yolov8: File exists
YOLOv8 setup complete.


In [13]:
# Download YOLOv7
%cd {eval_results_dir}
# !git clone https://github.com/WongKinYiu/yolov7
%cd yolov7
!pip install -r requirements.txt

from IPython import display
display.clear_output()
print('YOLOv7 setup complete.')

YOLOv7 setup complete.


In [14]:
# Download YOLOv5
%cd {eval_results_dir}
# !git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5

!pip install -qr requirements.txt  
import torch

from IPython import display
display.clear_output()
print('YOLOv5 setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

YOLOv5 setup complete. Using torch 2.0.0 CPU


In [15]:
%cd {eval_results_dir}

/Users/florenciavela/Desktop/flor/git/gun-detection/003-Evaluating-models


# Evaluate vs original datasets

In [39]:
total = len(os.listdir(train_results_dir))
counter = 0
for folder_name in sorted(os.listdir(train_results_dir)):
    counter += 1
    if not folder_name.startswith("model"):
        continue

    folder_path = os.path.join(train_results_dir, folder_name, "weights")
    print(f"{counter}/{total}")

    # Split the folder name into variables
    variables = folder_name.split('_')

    # Extract the required variables
    model = variables[1]
    tf = variables[3]
    imgsz = variables[5]
    epochs = variables[7]
    batch = variables[9]
    dataset = variables[11]
    loss = variables[13]
    lr = variables[15]
    
    print(folder_name)
    print("Model:", model)
    print("TF:", tf)
    print("Imgsz:", imgsz)
    print("Epochs:", epochs)
    print("Batch:", batch)
    print("Dataset:", dataset)
    print("Executing command...")
    print()
    
    if dataset == "v1":
        dataset_location = dataset_dir + model + "_original"
    elif dataset == "v2":
        dataset_location = dataset_dir + model + "_original_v2"
        
        
    command = None
    
    if model == "yolov5" and dataset in ["v1", "v2"]:
        best_pt_path = os.path.join(folder_path, "best.pt")
        if os.path.exists(best_pt_path):
            %cd yolov5
            command = f"python3 val.py --weights {best_pt_path} --data {dataset_location}/data.yaml --task test --name {folder_name}"
            print(command)

    elif model == "yolov7" and dataset in ["v1", "v2"]:
        best_pt_path = os.path.join(folder_path, "best.pt")
        if os.path.exists(best_pt_path):
            %cd yolov7
            command = f"python3 test.py --weights {best_pt_path} --data {dataset_location}/data.yaml --task test --name {folder_name}"
            print(command)

    elif model == "yolov8" and dataset in ["v1", "v2"]:
        best_pt_path = os.path.join(folder_path, "best.pt")
        if os.path.exists(best_pt_path):
            %cd yolov8
            command = f"yolo val model={best_pt_path} data={dataset_location}/data.yaml split=test name={folder_name}"
            print(command)

    if command:
        # Run the command using subprocess.Popen and capture the output
        with subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True) as process:
            # Open the output file for writing
            with open(f"{eval_results_dir}/{output_file}", "a") as file:
                file.write(folder_name)
                file.write(f"\nModel: {model}")
                file.write(f"\nTF: {tf}")
                file.write(f"\nImgsz: {imgsz}")
                file.write(f"\nEpochs: {epochs}")
                file.write(f"\nBatch: {batch}")
                file.write(f"\nDataset: {dataset}")
                file.write(f"\nLoss: {loss}")
                file.write(f"\nLr0: {lr}\n")
                # Read and display each line of the output as it becomes available
                for line in process.stdout:
                    print(line, end="")
                    file.write(line)
        %cd ..

    from IPython import display
    display.clear_output()
    

print("Executing commands...", "DONE!")
print()

Executing commands... DONE!



## Convert the logs to CSV

In [40]:
eval_results_dir = os.getcwd()
dataset_dir = os.getcwd()+"/../datasets/"

def process_file(file_path):
    results = list()
    curr_res = dict()

    with open(file_path, 'r') as file:
        lines = file.readlines()

    for line in lines:
        if line.startswith("Model:"):
            curr_res["model"] = line.split()[1]
        elif line.startswith("TF:"):
            curr_res["tf"] = line.split()[1]
        elif line.startswith("Imgsz:"):
            curr_res["imgsz"] = line.split()[1]
        elif line.startswith("Epochs:"):
            curr_res["epochs"] = line.split()[1]
        elif line.startswith("Batch:"):
            curr_res["batch"] = line.split()[1]
        elif line.startswith("Dataset:"):
            curr_res["dataset"] = line.split()[1]
        elif line.startswith("Loss:"):
            curr_res["loss_function"] = line.split()[1]
        elif line.startswith("Lr0:"):
            try:
                curr_res["lr0"] = float("0."+line.split()[1])
            except:
                curr_res["lr0"] = float(line.split()[1])
            
        if line.strip(" ").startswith("all"):
            P, R, mAP5, mAP95 = line.split()[3:]
            curr_res["all_P"] = P
            curr_res["all_R"] = R
            curr_res["all_F1"] = (float(P) * float(R)) / (float(P) + float(R))
            curr_res["all_mAP@.5"] = mAP5
            curr_res["all_mAP@.5:.95"] = mAP95

        elif line.strip(" ").startswith("knife"):
            P, R, mAP5, mAP95 = line.split()[3:]
            curr_res["knife_P"] = P
            curr_res["knife_R"] = R
            curr_res["knife_F1"] = (float(P) * float(R)) / (float(P) + float(R))
            curr_res["knife_mAP@.5"] = mAP5
            curr_res["knife_mAP@.5:.95"] = mAP95

        elif line.strip(" ").startswith("pistol") or line.strip(" ").startswith("gun"):
            P, R, mAP5, mAP95 = line.split()[3:]
            curr_res["pistol_P"] = P
            curr_res["pistol_R"] = R
            curr_res["pistol_F1"] = (float(P) * float(R)) / (float(P) + float(R))
            curr_res["pistol_mAP@.5"] = mAP5
            curr_res["pistol_mAP@.5:.95"] = mAP95

        elif line.strip(" ").startswith("Results saved to"):
            model_key = line.split("/")[-1].replace('\x1b[0m', '').replace('\n', '')
            model_info = model_key.split("_")

            curr_res["model_key"] = model_key

            results.append(curr_res)
            curr_res = dict()

    df = pd.DataFrame(results)
    df.set_index('model_key', inplace=True)
    return df

file_path = os.path.join(eval_results_dir, output_file)
df = process_file(file_path)
df.to_csv(f'{eval_results_dir}/output_csvs/results_val_vs_original_dataset.csv')
df

,model,tf,imgsz,epochs,batch,dataset,loss_function,lr0,all_P,all_R,...,knife_P,knife_R,knife_F1,knife_mAP@.5,knife_mAP@.5:.95,pistol_P,pistol_R,pistol_F1,pistol_mAP@.5,pistol_mAP@.5:.95
model_key,,,,,,,,,,,,,,,,,,,,,
model_yolov5_tf_no_imgsz_800_epochs_50_batch_16_dataset_v1_loss_SGD_lr_01,yolov5,no,800,50,16,v1,SGD,0.010,0.737,0.625,...,0.767,0.734,0.375069,0.795,0.443,0.707,0.517,0.298627,0.607,0.272
model_yolov5_tf_yolov5s_imgsz_640_epochs_100_batch_16_dataset_v1_loss_SGD_lr_001,yolov5,yolov5s,640,100,16,v1,SGD,0.001,0.692,0.603,...,0.759,0.715,0.368172,0.777,0.416,0.626,0.491,0.275171,0.552,0.249
model_yolov5_tf_yolov5s_imgsz_640_epochs_100_batch_16_dataset_v1_loss_SGD_lr_01,yolov5,yolov5s,640,100,16,v1,SGD,0.010,0.811,0.75,...,0.858,0.848,0.426485,0.89,0.548,0.765,0.653,0.352288,0.724,0.405
model_yolov5_tf_yolov5s_imgsz_640_epochs_50_batch_16_dataset_v1_loss_SGD_lr_01,yolov5,yolov5s,640,50,16,v1,SGD,0.010,0.773,0.712,...,0.83,0.822,0.412990,0.863,0.504,0.715,0.603,0.327121,0.666,0.355
model_yolov5_tf_yolov5s_imgsz_800_epochs_100_batch_16_dataset_v1_loss_SGD_lr_01,yolov5,yolov5s,800,100,16,v1,SGD,0.010,0.854,0.745,...,0.911,0.862,0.442911,0.901,0.575,0.796,0.628,0.351045,0.714,0.405
model_yolov5_tf_yolov5s_imgsz_800_epochs_50_batch_16_dataset_v1_loss_SGD_lr_01,yolov5,yolov5s,800,50,16,v1,SGD,0.010,0.796,0.716,...,0.876,0.855,0.432686,0.872,0.513,0.716,0.577,0.319514,0.638,0.343
model_yolov7_tf_yolov7training_imgsz_640_epochs_100_batch_16_dataset_v1_loss_SGD_lr_001,yolov7,yolov7training,640,100,16,v1,SGD,0.001,0.87,0.904,...,0.923,0.953,0.468880,0.958,0.687,0.816,0.855,0.417522,0.87,0.593
model_yolov7_tf_yolov7training_imgsz_640_epochs_100_batch_16_dataset_v1_loss_SGD_lr_01,yolov7,yolov7training,640,100,16,v1,SGD,0.010,0.874,0.849,...,0.934,0.924,0.464487,0.955,0.666,0.814,0.774,0.396748,0.806,0.581
model_yolov7_tf_yolov7training_imgsz_640_epochs_50_batch_16_dataset_v1_loss_SGD_lr_001,yolov7,yolov7training,640,50,16,v1,SGD,0.001,0.839,0.902,...,0.904,0.953,0.463927,0.956,0.664,0.774,0.85,0.405111,0.858,0.588


In [46]:
df[df.dataset == 'v2']

,model,tf,imgsz,epochs,batch,dataset,loss_function,lr0,all_P,all_R,...,knife_P,knife_R,knife_F1,knife_mAP@.5,knife_mAP@.5:.95,pistol_P,pistol_R,pistol_F1,pistol_mAP@.5,pistol_mAP@.5:.95
model_key,,,,,,,,,,,,,,,,,,,,,
model_yolov8_tf_yolov8m_imgsz_640_epochs_100_batch_16_dataset_v2_loss_SGD_lr_01,yolov8,yolov8m,640,100,16,v2,SGD,0.01,0.993,0.962,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
model_yolov8_tf_yolov8m_imgsz_640_epochs_50_batch_16_dataset_v2_loss_SGD_lr_01,yolov8,yolov8m,640,50,16,v2,SGD,0.01,0.98,0.962,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
model_yolov8_tf_yolov8s_imgsz_640_epochs_300_batch_16_dataset_v2_loss_SGD_lr_01,yolov8,yolov8s,640,300,16,v2,SGD,0.01,1,0.961,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
model_yolov8_tf_yolov8s_imgsz_800_epochs_100_batch_16_dataset_v2_loss_SGD_lr_01,yolov8,yolov8s,800,100,16,v2,SGD,0.01,0.987,0.961,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
